In [384]:
import pandas as pd
import os
from datasketch import MinHash
from sklearn.cross_validation import train_test_split;
import numpy as np
import math
import itertools
import random

In [80]:
os.listdir()

['.ipynb_checkpoints',
 'analysis.ipynb',
 'data.csv',
 'LSH.ipynb',
 'Michigan.xlsx']

In [81]:
AWS = weather['Average Wind Speed']

In [82]:
PRCP = weather['PRCP']

In [83]:
len(PRCP)

2910

In [90]:
df = pd.read_csv('data.csv')

In [91]:
df.fillna(0, inplace=True)

In [93]:
df.describe()

,Average Wind Speed,PRCP,TMAX,TMIN,Fastest 2 minute wind speed,Fastest 5 second wind speed,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
count,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000
mean,7.274945,0.079595,59.249828,37.227835,18.562715,28.019416,0.344674,0.100344,0.053265,0.150172,0.014433
std,3.533600,0.225217,21.368136,18.345112,7.288373,17.480074,0.475343,0.300509,0.224599,0.357301,0.119288
min,0.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.700000,0.000000,41.000000,25.000000,14.100000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.930000,0.000000,62.000000,38.000000,17.000000,23.900000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.400000,0.030000,78.000000,52.000000,21.900000,31.100000,1.000000,0.000000,0.000000,0.000000,0.000000
max,22.150000,4.000000,102.000000,73.000000,83.000000,185.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [215]:
AWS = df['Average Wind Speed']
AWS.std()

3.533599814994269

In [216]:
def splitVectors(array, maxVal, nSplits):
    featureVec = []
    for val in array:
        innerArr = [0] * (int(maxVal)//nSplits + 1)
        index = int(math.ceil(val/nSplits)) - 1
        innerArr[index] = 1
        featureVec.append(innerArr)
    return featureVec

In [217]:
AWS.max()

22.149999999999999

In [218]:
df.head()

,DATE,Average Wind Speed,PRCP,TMAX,TMIN,Fastest 2 minute wind speed,Fastest 5 second wind speed,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
0,1/1/2010,10.74,0.0,26.0,14.0,21.9,28.0,0.0,0.0,0.0,0.0,0.0
1,1/2/2010,11.18,0.0,14.0,5.0,23.9,35.1,0.0,0.0,0.0,0.0,0.0
2,1/3/2010,12.30,0.0,23.0,4.0,23.0,30.0,0.0,0.0,0.0,0.0,0.0
3,1/4/2010,9.40,0.0,25.0,19.0,16.1,21.9,0.0,0.0,0.0,1.0,0.0
4,1/5/2010,8.50,0.0,28.0,22.0,15.0,21.9,1.0,0.0,0.0,1.0,0.0


In [238]:
AWSvec = splitVectors(AWS.tolist(), AWS.max(), 3)

In [240]:
TMAXvec = splitVectors(df['TMAX'].tolist(), df['TMAX'].max(), 21)

In [230]:
FFSW = splitVectors(df['Fastest 5 second wind speed'].tolist(), df['Fastest 5 second wind speed'].max(), 17)

In [236]:
FTMW = splitVectors(df['Fastest 2 minute wind speed'].tolist(), df['Fastest 2 minute wind speed'].max(), 7)

In [252]:
Fog = df['Fog/Ice'].astype(int).tolist()

In [249]:
Thunder = df['Thunder'].astype(int).tolist()

In [251]:
PowerOutage = df['Power Outage'].astype(int).tolist()

In [277]:
featureVec = []

In [292]:
featureVec.append(AWSvec)
featureVec.append(TMAXvec)
featureVec.append(FFSW)
featureVec.append(FTMW)
featureVec.append(Fog)
featureVec.append(Thunder)
featureVec.append(PowerOutage)

In [282]:
len(featureVec)

7

In [309]:
featureVec = []
for i in range(len(FFSW)):
    innerVec = []
    innerVec.append(AWSvec[i])
    innerVec.append(TMAXvec[i])
    innerVec.append(FFSW[i])
    innerVec.append(FTMW[i])
    innerVec.append([Fog[i]])
    innerVec.append([Thunder[i]])
    innerVec.append([PowerOutage[i]])
    combined = [item for sublist in innerVec for item in sublist]
    featureVec.append(combined)

In [311]:
from datasketch import MinHash

In [321]:
m1 = MinHash()
m2 = MinHash()

In [319]:
for val in featureVec[0]:
    m1.update(str(val).encode('utf-8'))

In [339]:
test = []
for i in range(100):
    m1 = MinHash()
    m1.update(str(featureVec[i]).encode('utf-8'))
    for j in range(i, 101):
        m2 = MinHash()
        m2.update(str(featureVec[j]).encode('utf-8'))
        if m1.jaccard(m2) == 1 and i != j:
            test.append((i,j))
#         test[(i,j)] = m1.jaccard(m2)

In [346]:
mtest = MinHash()
mtest.update(str(featureVec[167]).encode('utf-8'))
mtest2 = MinHash()
mtest2.update(str(featureVec[194]).encode('utf-8'))

In [347]:
mtest.jaccard(mtest2)

0.0

In [348]:
pwr = []
for i in range(len(PowerOutage)):
    if PowerOutage[i] == 1:
        pwr.append(i)

In [368]:
test = []
for i in range(len(pwr) - 1):
    m1 = MinHash()
    m1.update(str(featureVec[pwr[i]]).encode('utf-8'))
    for j in range(i, len(pwr)):
        m2 = MinHash()
        m2.update(str(featureVec[pwr[j]]).encode('utf-8'))
        if m1.jaccard(m2) == 1 and i != j:
            test.append((pwr[i],pwr[j]))
#         test[(i,j)] = m1.jaccard(m2)

In [369]:
test

[(167, 608), (228, 907), (545, 1344), (1662, 2369)]

In [374]:
df.iloc[1662]

DATE                           7/27/2014
Average Wind Speed                  7.38
PRCP                                0.51
TMAX                                  88
TMIN                                  58
Fastest 2 minute wind speed           32
Fastest 5 second wind speed           49
Fog/Ice                                0
Heavy/Freezing Fog                     0
Thunder                                0
Smoke/Haze                             0
Power Outage                           1
Name: 1662, dtype: object

In [375]:
df.iloc[2369]

DATE                           7/8/2016
Average Wind Speed                 6.71
PRCP                               0.29
TMAX                                 89
TMIN                                 62
Fastest 2 minute wind speed          32
Fastest 5 second wind speed          38
Fog/Ice                               0
Heavy/Freezing Fog                    0
Thunder                               0
Smoke/Haze                            0
Power Outage                          1
Name: 2369, dtype: object

In [376]:
def getRandomIndexes(maxLength, numberIterations=10):
    """
    Input: Max length of the classes generated and number of hashes in the LSH
    Output: Get 10 pairs of random values
    """
    randoms = []
    for i in range(numberIterations):
        a = random.randint(0, maxLength)
        b = random.randint(0, maxLength)
        randoms.append([a,b])
    return randoms

In [400]:
def minHashFunction(indSet, randoms):
    """
    Input: The set whose hash function is to be found
        indSet: Independent Set - The set on which we wish to apply the LSH
        buckets: currently unused - will be the number of times we wish to apply minhash or the length of minhash
        classes: list of words available
        randoms: a,b pair of random numbers
    Output: Gives the minhash of the set
    """
    # c should be changed to the lowest prime that comes after the length of classes
    # this would ensure that the hashes are uniformly spread
    c = 41
    # c+1 because it is the next number after the greatest prime
    # all possible insertions are always less than this
    # len(randoms) = the number of iterations
    finalHash = [c+1] * len(randoms)
    for i in range(len(randoms)):
        # iterate  over the set
        for j, val in enumerate(indSet):
            # if the value is 1, find the minhash
            # a = randoms[i][0]; b = randoms[i][1]
            # minhash = (a*x + b) & c
            if val == 1:
                temp = (randoms[i][0]*j + randoms[i][1]) % c
#                 print(temp, finalHash[i])
                # track the smallest/minimum
                if temp < finalHash[i]:
                    finalHash[i] = temp
    return finalHash

In [391]:
def jaccardSimilarity(a, b):
    """
    Input - two sets - a and b
    Output - the jaccardian similarity given as a(intersection)b / a(union)b
    Gives a value to compare how similar two sets are
    """
    x = set(a)
    y = set(b)
    num = len(x.intersection(y))
    den = len(x.union(y))
    return num/den

In [382]:
len(featureVec[0])

39

In [408]:
listRandoms = getRandomIndexes(39, 10)

In [409]:
len(featureVec[100])

39

In [410]:
comp = []
for i in range(9):
    a = minHashFunction(featureVec[i], listRandoms)
    for j in range(i, 10):
        b = minHashFunction(featureVec[j], listRandoms)
        comp.append(jaccardSimilarity(a, b))

In [411]:
comp

[1.0,
 0.16666666666666666,
 0.4,
 0.7142857142857143,
 0.4,
 0.4,
 0.3,
 0.7142857142857143,
 0.2727272727272727,
 0.4,
 1.0,
 0.23076923076923078,
 0.16666666666666666,
 0.45454545454545453,
 0.45454545454545453,
 0.36363636363636365,
 0.16666666666666666,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 0.4,
 0.3333333333333333,
 0.3333333333333333,
 0.36363636363636365,
 0.4,
 0.23076923076923078,
 0.45454545454545453,
 1.0,
 0.4,
 0.4,
 0.3,
 1.0,
 0.4,
 0.4,
 1.0,
 1.0,
 0.5,
 0.4,
 0.45454545454545453,
 0.7777777777777778,
 1.0,
 0.5,
 0.4,
 0.45454545454545453,
 0.7777777777777778,
 1.0,
 0.3,
 0.5,
 0.6666666666666666,
 1.0,
 0.4,
 0.4,
 1.0,
 0.45454545454545453]